**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-zuhkfde8
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-zuhkfde8
  Resolved https://github.com/Kaggle/learntools.git to commit 93cf102cac95c2d8d7fb96ccc00ab5acc2c4bf05
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268981 sha256=49e46af5a31639270bc2644b7a61a7439dc6313048ca53d35440e9d52d364b02
  Stored in directory: /tmp/pip-ephem-wheel-cache-ibknkslk/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,968b34f43738f7b1e7542be85d16f36d5ef43197,e17e0bd91a11c26e916a8de5bee31ef2a503043b17ea4c...,2023-06-15 04:30:00+00:00,2023-06-15 04:45:00+00:00,689,11.25,NaN,NaN,53,68,...,0.0,28.75,Prcard,Flash Cab,41.673820,-87.635740,POINT (-87.6357397768 41.6738199043),41.777196,-87.642498,POINT (-87.6424975271 41.7771962546)
1,94ee0d74633f1608f1272026ecb0192301da3cf6,27ac571a7da50a8a926db5a27583ef282a290e500ad21c...,2023-05-19 14:00:00+00:00,2023-05-19 14:15:00+00:00,1260,4.70,NaN,NaN,44,39,...,0.0,15.75,Unknown,Taxi Affiliation Services,41.740206,-87.615970,POINT (-87.6159695226 41.7402057565),41.808916,-87.596183,POINT (-87.5961833442 41.8089162826)
2,952d18688bbf1baa313f0ceb8d17dde0073ff88e,a46aedb114a720df392300b351493f1a65cd0e8f485f70...,2023-05-12 12:45:00+00:00,2023-05-12 12:45:00+00:00,134,0.49,NaN,NaN,38,39,...,0.0,4.50,Cash,Flash Cab,41.812949,-87.617860,POINT (-87.6178596758 41.8129489392),41.808916,-87.596183,POINT (-87.5961833442 41.8089162826)
3,969c6aa4e7b25eed03da7f1afe4301e8719b14b1,99a996d079da15de20cb43da904ab1dfa54610da249cf2...,2023-06-27 16:45:00+00:00,2023-06-27 17:30:00+00:00,1815,14.26,NaN,NaN,75,36,...,0.0,37.00,Prcard,Flash Cab,41.689730,-87.669054,POINT (-87.6690544032 41.6897299145),41.823713,-87.602350,POINT (-87.6023504372 41.8237128095)
4,96713e31c63320ec82fb5de005c1ccd7a0ebce77,1b84d02f4f9b66792e2b5fb36d19ec7f5a4eb96f26ed6c...,2023-05-31 13:15:00+00:00,2023-05-31 14:15:00+00:00,3635,6.58,NaN,NaN,69,39,...,0.0,29.50,Prcard,City Service,41.763247,-87.616134,POINT (-87.6161341112 41.7632467988),41.808916,-87.596183,POINT (-87.5961833442 41.8089162826)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [11]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-01-09,79842.000000
1,2016-01-04,74260.571429
2,2016-02-12,93339.428571
3,2016-02-29,92275.285714
4,2016-03-03,93109.857143


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [12]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [18]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() OVER (
                            PARTITION BY pickup_community_area
                            ORDER BY trip_start_timestamp
                        ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,1.0,2013-10-03 00:00:00+00:00,2013-10-03 00:15:00+00:00,1
1,1.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
2,1.0,2013-10-03 00:30:00+00:00,2013-10-03 00:45:00+00:00,3
3,1.0,2013-10-03 00:30:00+00:00,2013-10-03 01:00:00+00:00,3
4,1.0,2013-10-03 00:30:00+00:00,2013-10-03 01:00:00+00:00,3


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [22]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp )
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp
                                ), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,145960422e0a08c023dc4616dcb30daeff190b424755b0...,2013-10-03 08:30:00+00:00,2013-10-03 08:30:00+00:00,285.0
1,15c03618b76c932fd23d3054627cdbbd21323bd13316f4...,2013-10-03 08:30:00+00:00,2013-10-03 08:45:00+00:00,330.0
2,15c03618b76c932fd23d3054627cdbbd21323bd13316f4...,2013-10-03 18:30:00+00:00,2013-10-03 19:15:00+00:00,240.0
3,342509ce9f04d14185cf073c4cd71ea079413a622ea3c2...,2013-10-03 07:00:00+00:00,2013-10-03 07:15:00+00:00,375.0
4,342509ce9f04d14185cf073c4cd71ea079413a622ea3c2...,2013-10-03 18:15:00+00:00,2013-10-03 18:15:00+00:00,405.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [21]:
# Lines below will give you a hint or solution code
q_3.hint()
#q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> The `TIMESTAMP_DIFF()` function takes three arguments, where the first (`trip_start_timestamp`) and the last (`MINUTE`) are provided for you.  This function provides the time difference (in minutes) of the timestamps in the first two arguments. You need only fill in the second argument, which should use the **LAG()** function to pull the timestamp corresponding to the end of the previous trip (for the same `taxi_id`).

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*